In [2]:
import torch
import sys

NUM_DATA = 500
t = torch.rand(NUM_DATA, dtype=torch.float32)
t.element_size() * t.numel()

2000

In [4]:
from ogb.nodeproppred import PygNodePropPredDataset
import torch_geometric.transforms as T
import os.path as osp

path = osp.join('/data/gangda', 'ogb')
dataset = PygNodePropPredDataset(name='ogbn-products', root=path, transform=T.ToSparseTensor())
data = dataset[0]

In [20]:
data.adj_t.set_value_(torch.rand(data.num_edges), layout='csc')

SparseTensor(row=tensor([      0,       0,       0,  ..., 2449028, 2449028, 2449028]),
             col=tensor([    384,    2412,    7554,  ..., 1787657, 1864057, 2430488]),
             val=tensor([0.3932, 0.8904, 0.9028,  ..., 0.4292, 0.5621, 0.6839]),
             size=(2449029, 2449029), nnz=123718280, density=0.00%)

In [22]:
from torch_geometric_autoscale import metis, permute, SubgraphLoader

perm, ptr = metis(data.adj_t, 4, log=True)
data_ = permute(data, perm, log=True)

Computing METIS partitioning with 4 parts... Done! [14.74s]
Permuting data... Done! [29.68s]


In [125]:
degree = data_.adj_t.sum(dim=1)
indptr, indices, value = data_.adj_t.csr()

In [134]:
i = 0
print(degree[i].item())
value[indptr[i]:indptr[i+1]].sum().item()

2.894808769226074


2.8948090076446533

In [75]:
data_list = list(SubgraphLoader(data_, ptr, batch_size=1, shuffle=False))
sub_data = data_list[0]

Pre-processing subgraphs... Done! [3.67s]


In [84]:
res = sub_data.data.adj_t.csr()
res[2][:6].sum()

tensor(2.8948)

In [86]:
adj = data.adj_t.t()
perm, ptr = metis(adj, 4, log=True)
data_ = permute(data, perm, log=True)

SparseTensor(row=tensor([      0,       0,       0,  ..., 2449028, 2449028, 2449028]),
             col=tensor([    384,    2412,    7554,  ..., 1787657, 1864057, 2430488]),
             val=tensor([0.7309, 0.3331, 0.4298,  ..., 0.7652, 0.4662, 0.5261]),
             size=(2449029, 2449029), nnz=123718280, density=0.00%)

In [85]:
data_.adj_t.sum(dim=1).to(torch.float)

tensor([ 2.8948,  8.1311,  5.3858,  ...,  4.2185, 61.1699, 19.1762])